In [2]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch import nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import f1_score

In [3]:
n_qubits = 2
n_layers = 2
noise_type = "amplitude_damping"   # options: "bit_flip","phase_flip","amplitude_damping","phase_damping","depolarizing", or None
noise_prob = 0.05
epochs = 30
lr = 0.05

In [4]:
# Load binary iris (first two classes)
iris = load_iris()
mask = iris.target < 2
X = iris.data[mask]
y = iris.target[mask].astype(float)  # {0,1}

# Use first n_qubits features if available, otherwise tile features to match n_qubits
X_raw = X
if X_raw.shape[1] >= n_qubits:
    X = X_raw[:, :n_qubits]
else:
    reps = int(np.ceil(n_qubits / X_raw.shape[1]))
    X = np.tile(X_raw, reps)[:, :n_qubits]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Device
dev = qml.device("default.mixed", wires=n_qubits)

In [5]:
def add_noise_all(prob):
    """Apply single-qubit noise channel to each wire (fixed)."""
    if noise_type is None or noise_type.lower() == "none":
        return
    for w in range(n_qubits):
        t = noise_type.lower()
        if t == "bit_flip":
            qml.BitFlip(prob, wires=w)
        elif t == "phase_flip":
            qml.PhaseFlip(prob, wires=w)
        elif t == "amplitude_damping":
            qml.AmplitudeDamping(prob, wires=w)
        elif t == "phase_damping":
            qml.PhaseDamping(prob, wires=w)
        elif t == "depolarizing":
            qml.DepolarizingChannel(prob, wires=w)
        else:
            raise ValueError(f"Unknown noise type: {noise_type}")

# helper to safe-embed input (tile/pad to n_qubits)
def angle_embed_inputs(x):
    vec = np.array(x)
    if vec.size < n_qubits:
        reps = int(np.ceil(n_qubits / vec.size))
        vec = np.tile(vec, reps)[:n_qubits]
    qml.AngleEmbedding(vec, wires=range(n_qubits))

@qml.qnode(dev, interface="autograd")
def circuit(inputs, weights):
    # inputs: length n_qubits (after padding)
    angle_embed_inputs(inputs)
    # Strongly entangling layers expect shape (n_layers, n_wires, 3)
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    add_noise_all(noise_prob)   # noise AFTER ansatz
    return qml.expval(qml.PauliZ(0))

# cost function (MSE between (exp+1)/2 and labels in {0,1})
def cost(weights, Xbatch, Ybatch):
    preds = []
    for x in Xbatch:
        ev = circuit(x, weights)        # in [-1,1]
        preds.append((ev + 1.0) / 2.0)  # map to [0,1]
    preds = np.array(preds)
    return np.mean((preds - Ybatch) ** 2)

# initialize weights with correct shape (n_layers, n_qubits, 3)
weights = np.random.uniform(0, 2 * np.pi, size=(n_layers, n_qubits, 3), requires_grad=True)

opt = qml.GradientDescentOptimizer(stepsize=lr)


In [6]:
# Training loop
for epoch in range(epochs):
    weights, train_loss = opt.step_and_cost(lambda w: cost(w, X_train, y_train), weights)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:3d} | Train Loss: {train_loss:.6f}")


Epoch   5 | Train Loss: 0.380990
Epoch  10 | Train Loss: 0.356965
Epoch  15 | Train Loss: 0.333840
Epoch  20 | Train Loss: 0.311900
Epoch  25 | Train Loss: 0.291392
Epoch  30 | Train Loss: 0.272494


In [7]:
# Evaluation
t0 = time.perf_counter()
preds_test = [ (circuit(x, weights) + 1.0) / 2.0 for x in X_test ]
t1 = time.perf_counter()
preds_test = np.array(preds_test).flatten()

# Threshold for binary classification
preds_binary = (preds_test > 0.5).astype(int)
y_true_int = np.array(y_test).astype(int)

# Metrics
mse = mean_squared_error(y_true_int, preds_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_int, preds_test)
f1 = f1_score(y_true_int, preds_binary)
inference_time_per_sample = (t1 - t0) / len(X_test)

print("\n=== SEA Evaluation ===")
print(f"Noise type: {noise_type}")
print(f"Noise prob: {noise_prob}")
print(f"# qubits: {n_qubits}")
print(f"MSE:  {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE:  {mae:.6f}")
print(f"F1-score:  {f1:.6f}")
print(f"Inference time per sample: {inference_time_per_sample*1000:.4f} ms")


=== SEA Evaluation ===
Noise type: amplitude_damping
Noise prob: 0.05
# qubits: 2
MSE:  0.303380
RMSE: 0.550799
MAE:  0.532466
F1-score:  0.454545
Inference time per sample: 4.1167 ms
